In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Unique data and Window Functions").getOrCreate()
)

In [2]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = ["emp_id", "dept_id", "emp_name", "age", "gender", "salary", "hire_date"]

In [3]:
# Create emp DataFrame
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [5]:
# Show emp dataframe (ACTION)
emp_df.show()

+------+-------+-------------+---+------+------+----------+
|emp_id|dept_id|     emp_name|age|gender|salary| hire_date|
+------+-------+-------------+---+------+------+----------+
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|   006|    103|    Jill Wong| 32|Female| 52000|2018-07-01|
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|
|   009|    103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|   011|    104|   David Park| 38|  Male| 65000|2015-11-01|
|   012|    105|   Susan Chen| 31|Female| 54000|2017-02-15|
|   013|    106|    Brian Kim| 45|  Male| 75000|2011-07-01|
|   014|    107|    Emily Lee| 26|Female

In [6]:
emp_df.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [7]:
# Get unique data
# select distinct emp.* from emp
emp_unique = emp_df.distinct()

In [8]:
emp_unique.show()

+------+-------+-------------+---+------+------+----------+
|emp_id|dept_id|     emp_name|age|gender|salary| hire_date|
+------+-------+-------------+---+------+------+----------+
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|   010|    104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|
|   009|    103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|   006|    103|    Jill Wong| 32|Female| 52000|2018-07-01|
|   014|    107|    Emily Lee| 26|Female| 46000|2019-01-01|
|   011|    104|   David Park| 38|  Male| 65000|2015-11-01|
|   013|    106|    Brian Kim| 45|  Male| 75000|2011-07-01|
|   012|    105|   Susan Chen| 31|Female

In [10]:
# Unique of department_ids
# select distinct department_id from emp
emp_dept_id = emp_df.select("dept_id").distinct()

In [11]:
emp_dept_id.show()

+-------+
|dept_id|
+-------+
|    101|
|    102|
|    103|
|    104|
|    107|
|    106|
|    105|
+-------+



In [12]:
# Window Functions
# select *, max(salary) over(partition by department_id order by salary desc) as max_salary from emp_unique
from pyspark.sql.window import Window
from pyspark.sql.functions import max, col, desc

window_spec = Window.partitionBy(col("dept_id")).orderBy(col("salary").desc())
max_func = max(col("salary")).over(window_spec)

emp_1 = emp_df.withColumn("max_salary", max_func)

In [13]:
emp_1.show()

+------+-------+-------------+---+------+------+----------+----------+
|emp_id|dept_id|     emp_name|age|gender|salary| hire_date|max_salary|
+------+-------+-------------+---+------+------+----------+----------+
|   007|    101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|
|   001|    101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|
|   002|    101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|
|   003|    102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|   020|    102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
|   008|    102|     Kate Kim| 29|Female| 51000|2019-10-01|     55000|
|   004|    102|    Alice Lee| 28|Female| 48000|2017-09-30|     55000|
|   019|    103|  Steven Chen| 36|      | 62000|2015-08-01|     62000|
|   005|    103|    Jack Chan| 40|  Male| 60000|2013-04-01|     62000|
|   009|    103|      Tom Tan| 33|  Male| 58000|2016-06-01|     62000|
|   006|    103|    Jill Wong| 32|Female| 52000|2018-07-01|     62000|
|   01

In [14]:
# Window Functions - 2nd highest salary of each department
# select *, row_number() over(partition by department_id order by salary desc) as rn from emp_unique where rn = 2
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col

window_spec = Window.partitionBy(col("dept_id")).orderBy(col("salary").desc())
rn = row_number().over(window_spec)

emp_2 = emp_df.withColumn("rn", rn).where("rn = 2")

In [15]:
emp_2.show()

+------+-------+-----------+---+------+------+----------+---+
|emp_id|dept_id|   emp_name|age|gender|salary| hire_date| rn|
+------+-------+-----------+---+------+------+----------+---+
|   001|    101|   John Doe| 30|  Male| 50000|2015-01-01|  2|
|   020|    102|  Grace Kim| 32|Female| 53000|2018-11-01|  2|
|   005|    103|  Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|   018|    104|  Nancy Liu| 29|Female| 50000|2017-06-01|  2|
|   012|    105| Susan Chen| 31|Female| 54000|2017-02-15|  2|
|   015|    106|Michael Lee| 37|  Male| 63000|2014-09-30|  2|
|   014|    107|  Emily Lee| 26|Female| 46000|2019-01-01|  2|
+------+-------+-----------+---+------+------+----------+---+



In [17]:
# Window function using expr
# select *, row_number() over(partition by department_id order by salary desc) as rn from emp_unique where rn = 2
from pyspark.sql.functions import expr

emp_3 = emp_df.withColumn("rn", expr("row_number() over(partition by dept_id order by salary desc)")).where("rn = 2")

In [18]:
emp_3.show()

+------+-------+-----------+---+------+------+----------+---+
|emp_id|dept_id|   emp_name|age|gender|salary| hire_date| rn|
+------+-------+-----------+---+------+------+----------+---+
|   001|    101|   John Doe| 30|  Male| 50000|2015-01-01|  2|
|   020|    102|  Grace Kim| 32|Female| 53000|2018-11-01|  2|
|   005|    103|  Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|   018|    104|  Nancy Liu| 29|Female| 50000|2017-06-01|  2|
|   012|    105| Susan Chen| 31|Female| 54000|2017-02-15|  2|
|   015|    106|Michael Lee| 37|  Male| 63000|2014-09-30|  2|
|   014|    107|  Emily Lee| 26|Female| 46000|2019-01-01|  2|
+------+-------+-----------+---+------+------+----------+---+



In [19]:
spark.stop()